# NovelD

In [8]:
from model.modules.lnoveld import NovelD

m = NovelD(10, 5, 32)

In [9]:
import torch

state = torch.rand(1, 10)
state

tensor([[0.0170, 0.7446, 0.7682, 0.4385, 0.1954, 0.8637, 0.0278, 0.0760, 0.4399,
         0.3038]])

In [10]:
m.get_reward(state)

{(0.017005085945129395, 0.7446469664573669, 0.7681810259819031, 0.43851786851882935, 0.19538533687591553, 0.863665759563446, 0.027759432792663574, 0.07602101564407349, 0.4399014711380005, 0.30379509925842285): 1}
tensor([[0.0170, 0.7446, 0.7682, 0.4385, 0.1954, 0.8637, 0.0278, 0.0760, 0.4399,
         0.3038]])
tensor([[-0.2554, -0.0441,  0.0087,  0.2480,  0.0232]])
tensor([[ 0.0338,  0.1748, -0.5254,  0.4180, -0.0263]],
       grad_fn=<AddmmBackward0>)
tensor([0.6694])
None
No reward


(0.0, tensor(0.0896, grad_fn=<MseLossBackward0>))

In [31]:
m.get_reward(torch.rand(2, 10))

TypeError: unhashable type: 'list'

In [23]:
r, l = m.get_reward(state)

{(0.017005085945129395, 0.7446469664573669, 0.7681810259819031, 0.43851786851882935, 0.19538533687591553, 0.863665759563446, 0.027759432792663574, 0.07602101564407349, 0.4399014711380005, 0.30379509925842285): 10, (0.7085840106010437, 0.31842857599258423, 0.5146272778511047, 0.7864795327186584, 0.47004234790802, 0.3820362091064453, 0.6211551427841187, 0.2838684320449829, 0.8736599087715149, 0.31877100467681885): 1}
tensor([[0.0170, 0.7446, 0.7682, 0.4385, 0.1954, 0.8637, 0.0278, 0.0760, 0.4399,
         0.3038]])
tensor([[-0.2554, -0.0441,  0.0087,  0.2480,  0.0232]])
tensor([[ 0.0063,  0.1617, -0.4729,  0.3883, -0.0200]],
       grad_fn=<AddmmBackward0>)
tensor([0.6036])
tensor([0.6099])
No reward


In [35]:
import numpy as np
a = np.array([[1,1]])
a + a * 0.1

array([[1.1, 1.1]])

# LNovelD

In [17]:
from model.modules.lnoveld import LNovelD

m = LNovelD(2, 2, 5)

In [18]:
m

LNovelD(
  (obs_noveld): NovelD(
    (target): MLPNetwork(
      (mlp): Sequential(
        (0): Linear(in_features=2, out_features=64, bias=True)
        (1): ReLU()
        (2): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
        (3): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
        (4): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
        (5): Linear(in_features=64, out_features=5, bias=True)
      )
    )
    (predictor): MLPNetwork(
      (mlp): Sequential(
        (0): Linear(in_features=2, out_features=64, bias=True)
        (1): ReLU()
        (2): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
        (3): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): ReLU()
        )
    

In [5]:
obs = torch.Tensor([[10.0, 10.0]])
m(obs, None)


target tensor([[-1.8384,  0.9695, -3.8054, -2.1500,  3.6722]])
pred tensor([[ 2.2023, -4.3319, -0.7381, -3.4516,  0.7183]],
       grad_fn=<AddmmBackward0>)
nov tensor([8.0162])
last nov tensor([0.8016])
comp tensor([-3.2065])
7.615405559539795


7.615405559539795

In [6]:
m.obs_noveld.episode_states_count

{(1.0, 1.0): 1, (10.0, 10.0): 1}